# A Semantic Analysis of the Voynich Manuscript using Spectral Clustering 

## Introduction

In this brief markdown-article thing, we're going to be investigating the semantic content of voynichese words using a method called _Spectral Clustering_. Spectral clustering is a clustering method that uses the _spectrum_ (eigenvalues and eigenvectors) of a, so called, _Normalized Laplacian_ matrix to cluster the data. A normalized laplacian matrix is a sort of representation of a graph, that encodes things like edge weights and node degree. The point of doing this is that selecting a subset of the spectrum to cluster on will reduce the dimensionality of the data, which makes the clustering more clear (see _curse of dimensionality_), while still retaining meaningful properties of the data. 

The goal is to find clusters of similar meaning words by using this clustering method on the data by structuring it in terms of a graph. Specifically, the nodes of this graph correspond to unique tokens in the text, and the edge weights correspond to the similarity in meaning of two words. This similarity is calculated by embedding each unique token using a word2vec model, and finding the cosine similarity between the embeddings. 

Hopefully, by using this technique we will find synonyms, words that belong to the same conceptual category, or perhaps grammatical particles that co-occur in similar contexts. 

## Data Sourcing

The text of the Voynich Manuscript has been digitized and is available online now-a-days. It has been digitized in a number of different ways, but the version we are going to be using comes from Luke Lindemann and Claire Bowern's github page, which they used in their paper _Character Entropy in Modern and Historical Texts: Comparison Metrics for an Undeciphered Manuscript_. Specifically, we are going to be using the Maximal Full version, which transcribes voynichese in greater detail. The reason we will be using this transcription is becuase if there is a difference between, for example, _q'o_ and _qo_ semantically, we sure would like to know about it. \[need to go into greater detail about origin of transcription schemes? writing is a little all over the place? need to explain where these texts come from in the first place?\]

## Methods

In this section, we will describe our application method of spectral clustering, and how we implemented it in Python. Specifically, we will show you the following:
- How we trained the word2vec model 
- How we created our adjacency matrix and normalized laplacian
- How we found the spectrum of the normalized laplacian matrix 
- How we found an opitimal number of clusters for the data. 
- How we clustered the resulting eigenvectors using Kmeans Clustering
- How to score each clustering model with a silhouette score
We will begin with loading in the texts and the training of the word2vec model.

### Data Wranlging and Word2Vec Model



\[writing TODO: describe which word2vec model, describe how it only takes in sentences, describe how we got the raw corpus data into sentences, and \]

\[coding TODO: consider how we might better get corpus into sentences, i.e. maybe 5 token strings are not best, find natural sentences?\]




In [1]:
import numpy as np
import gensim
from gensim.models import Word2Vec
import scipy

path = """/Users/karlmudsam/Desktop/Voynich_Spectral_Clustering/Voynich_A_Maximal_Text.txt"""

#loading in text file
voynich_text_file = open(path)
voynich_text = voynich_text_file.read()
voynich_tokenized = voynich_text.split()
voynich_tokenized = [i for i in voynich_tokenized if i.isalpha()]

voynich_sentences = []

#Turn tokenization into sentences, because that's what gensim's word2vec model wants
i = 0
while i < len(voynich_tokenized):
    sentence = voynich_tokenized[i:i+5]
    voynich_sentences.append(sentence)
    i = i + 5

#getting all the unique tokens
voynich_vocab = list(set(voynich_tokenized))

#creating the word2vec model
model = gensim.models.Word2Vec(voynich_sentences, min_count = 1, vector_size = 100, window = 5)

### Adjacency Matrix

Once we have a trained word2vec model, and a list of all the unique tokens, we can create a numpy matrix object and populate it with our edge weights. We do this by using gensim's  `model.wv.similarity` function. Because we do not want self-loops in our graph, whenever we attempt to find the similarity of a word with itself we  fill it in with a 0. 

The formula for a normalized laplacian is:
$$
L^{\text{norm}} = I - D^{-1/2}AD^{-1/2}
$$
Where:
- $L^{\text{norm}}$ is the normalized laplacian
- $D^{-1/2}$ is the matrix square-root of the degree matrix
- $A$ is the adjacency matrix
- $I$ is the identity matrix

Numpy's linear algebra library makes this a dream; all you have to do is apply the appropriate functions, and do some matrix algebra! 

In [3]:
#initializing similarity matrix
similarity_matrix = np.matrix(np.zeros( (len(voynich_vocab), len(voynich_vocab))) )

#populating similarity matrix, excluding diagonal
for i in range(len(voynich_vocab)):
    for j in range(len(voynich_vocab)):
        if i == j:
            similarity_matrix[i,j] = 0
        else:
            similarity_matrix[i,j] = model.wv.similarity(voynich_vocab[i], voynich_vocab[j])

#intializing degree matrix
degree_matrix = np.matrix(np.zeros( (len(voynich_vocab), len(voynich_vocab))) )

#little linear algebra trick for ya'll, summing the columns withh 
column_sums = similarity_matrix.sum(axis = 0)

#populating degree matrix
for j in range(len(voynich_vocab)):
    degree_matrix[j,j] = column_sums[0,j]


#Creating the normalized laplacian

#setup, finding all the terms that go into the normalized laplacian
degree_matrix_inverse = np.linalg.inv(degree_matrix)
degree_matrix_inv_sqrt = scipy.linalg.sqrtm(degree_matrix_inverse)
D = np.real(degree_matrix_inv_sqrt)
A = similarity_matrix
I = np.identity(len(voynich_vocab))

#actually doing the computation to find the normalized laplacian
L = I - (D@(A@D))

### Eigendecomposition of Normalized Laplacian Matrix

Again, Numpy has a great linear algebra library, which allows us to find the eigenvalues and eigenvectors of our normalized laplacian very easily, using `linalg.eig`. After we have obtained our eigenvectors and eigenvalues, we want to order the eigenvectors by the size of their associated eigenvalue. This will allow us to retain most of the _information_ \[incorrect word?\] from the original data when we truncate the eigenvector matrix, _as the eigenvectors with lower eigenvalues contain more information_ \[untrue?, oversimplification?\].

In [ ]:
#finding the eigenvectors and eigenvalues
eig = np.linalg.eig(L)

#sorting 'em by eigenvales, because the eigenvectors of the smallest eigenvalues have the greatest influence in clustering
zipped_vals_and_vects = zip(eig[0], eig[1])

#Sort these 2-tuples by their eigenvalues
sorted_vals_and_vects = sorted(zipped_vals_and_vects, key = lambda x: x[0])

#Create a new eigenvector array, because turning this zipped list into a numpy matrix will be a pain
eigenvector_matrix = np.zeros(shape = (len(voynich_vocab), len(voynich_vocab)))

#populate the eigenvector array
for i in range(len(sorted_vals_and_vects)):
    eigenvector_matrix[i] = sorted_vals_and_vects[i][1]

#Turn it into a numpy matrix
eigenvector_matrix = np.matrix(eigenvector_matrix)

#Transpose, so that we'll be able to 
eigenvector_matrix = eigenvector_matrix.T

#We are using a subset of eigenvectors to cluster to take advantage of dimensionality reduction
truncated_eigenvector_matrix = eigenvector_matrix[:,0:20] 

### Finding Optimal Number of Clusters
Finally we can start clustering! But how do we find the optimal number of clusters for our data? This question requires a metric for how good a certain clustering model is --and while there are a few-- we are going to use _Silhouette Scores_. Put briefly, a silhouette score can tell you on average how well a data point fits it's own cluster and how poorly it fits other clusters. It is scored on  a scale of -1 to 1, with 1 having the best fit and -1 having the worst. I'm not going to explain how to calculate them, but the Wikipedia page for silhouette scores explains the method of calculating them, and the Python package _sklearn_ has a good implementation of it. That, however, is a tangent. The plan then is simple, we form a clustering model for $i$ clusters, and the model with $i$ clusters and the greatest silhouette score is the optimal number of clusters. In the code below we calculate the silhouette score for models with up to 300 clusters.



In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score


scores = []
max_clusters = 300
for i in range(max_clusters - 1):
    print("Number of Clusters: ", i)
    kmeans = KMeans(n_clusters=i+2, random_state = 303).fit(truncated_eigenvector_matrix) #Training
    labels = kmeans.labels_
        
    score = silhouette_score(truncated_eigenvector_matrix, labels) #Calculating silhouette score
    scores.append(score)

num_clusters = np.arange(2,max_clusters+1) #Start at 2 because 1 cluster is trivial

plt.plot(num_clusters, scores)
plt.title("Silhouette Score vs Cluster Count in the \nComplete Voynich Manuscript")
plt.xlabel("Cluster Count (n)")
plt.ylabel("Silhouette Score")
plt.ylim(ymin = 0, ymax = 0.6)
plt.show()

## Results

## Discussion of Results

## Further Investigation

## Conclusion